In [319]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ast import literal_eval
import re
import nltk
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [320]:
df_anime = pd.read_csv('Anime_data.csv', low_memory=False)

In [321]:
df_anime = df_anime[:10000]

In [322]:
df_anime.describe()

,Anime_id,Rating,ScoredBy,Popularity,Members,Episodes
count,10000.000000,9986.000000,9.986000e+03,9926.000000,1.000000e+04,9933.000000
mean,12546.539400,6.440038,1.366161e+04,7333.624018,2.648652e+04,13.173462
std,10582.190672,0.990470,4.834723e+04,4315.251361,8.313500e+04,49.416582
min,1.000000,1.900000,1.000000e+00,1.000000,2.000000e+00,1.000000
25%,2964.500000,5.802500,9.600000e+01,3610.250000,3.740000e+02,1.000000
50%,9006.500000,6.500000,8.125000e+02,7279.000000,2.165000e+03,2.000000
75%,22059.000000,7.160000,5.719500e+03,10948.750000,1.318700e+04,12.000000
max,31994.000000,9.250000,1.006242e+06,15637.000000,1.451708e+06,1818.000000


In [323]:
def missing_values(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns={
        0: 'Missing Values',
        1: '% of Total Values'
    })
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
            '% of Total Values', ascending=False).round(1)
    print("Dataframe has " + str(df.shape[1]) + " columns.")
    print("There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")
    
    return mis_val_table_ren_columns

In [324]:
miss_values = missing_values(df_anime)
miss_values

Dataframe has 15 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
Producer,3857,38.6
Studio,3710,37.1
Synopsis,339,3.4
Popularity,74,0.7
Aired,74,0.7
Link,74,0.7
Episodes,67,0.7
Genre,29,0.3
Rating,14,0.1
ScoredBy,14,0.1


In [325]:
#drop missing value where rating is missing

df_anime = df_anime.dropna(subset=["Rating"])

In [326]:
#fill missing values using the mean

df_anime['ScoredBy'] = df_anime['ScoredBy'].fillna(df_anime['ScoredBy'].median())
df_anime['Popularity'] = df_anime['Popularity'].fillna(df_anime['Popularity'].median())
df_anime['Members'] = df_anime['Members'].fillna(df_anime['Members'].median())

In [327]:
#drop useless column

df_anime = df_anime.drop('Aired', axis=1)
df_anime = df_anime.drop('Anime_id', axis=1)
df_anime = df_anime.drop('Link', axis=1)
df_anime = df_anime.drop('Source', axis=1)
df_anime = df_anime.drop('Episodes', axis=1)
df_anime = df_anime.drop('Popularity', axis=1)
df_anime = df_anime.drop('ScoredBy', axis=1)
df_anime = df_anime.drop('Members', axis=1)

In [328]:
df_anime

,Title,Genre,Synopsis,Type,Producer,Studio,Rating
0,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81
1,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41
2,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31
3,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34
4,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04
...,...,...,...,...,...,...,...
9995,Okusama ga Seitokaichou!: Seitokaichou to Ofur...,"['Comedy', 'Romance', 'Ecchi', 'Shounen']",Bundled with the limited edition ninth manga v...,OVA,NaN,['Seven'],6.89
9996,Lupin Shanshei Pilot,"['Parody', 'Comedy']",NaN,Special,NaN,['TMS Entertainment'],5.69
9997,Hibike! Euphonium 2,"['Music', 'School', 'Drama']",Following their success in the qualifying roun...,TV,"['Lantis', 'Pony Canyon', 'Rakuonsha']",['Kyoto Animation'],8.29
9998,Hibike! Euphonium Movie 1: Kitauji Koukou Suis...,"['Music', 'School']",After swearing off music due to an incident at...,Movie,"['Lantis', 'Shochiku']",['Kyoto Animation'],7.45


In [329]:
miss_values = missing_values(df_anime)
miss_values

Dataframe has 7 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
Producer,3846,38.5
Studio,3704,37.1
Synopsis,338,3.4
Genre,29,0.3


In [330]:
df_anime['Synopsis'] = df_anime['Synopsis'].fillna("unknown")

In [331]:
df_anime

,Title,Genre,Synopsis,Type,Producer,Studio,Rating
0,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81
1,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41
2,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31
3,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34
4,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04
...,...,...,...,...,...,...,...
9995,Okusama ga Seitokaichou!: Seitokaichou to Ofur...,"['Comedy', 'Romance', 'Ecchi', 'Shounen']",Bundled with the limited edition ninth manga v...,OVA,NaN,['Seven'],6.89
9996,Lupin Shanshei Pilot,"['Parody', 'Comedy']",unknown,Special,NaN,['TMS Entertainment'],5.69
9997,Hibike! Euphonium 2,"['Music', 'School', 'Drama']",Following their success in the qualifying roun...,TV,"['Lantis', 'Pony Canyon', 'Rakuonsha']",['Kyoto Animation'],8.29
9998,Hibike! Euphonium Movie 1: Kitauji Koukou Suis...,"['Music', 'School']",After swearing off music due to an incident at...,Movie,"['Lantis', 'Shochiku']",['Kyoto Animation'],7.45


# Column "Genre"

In [332]:
df_anime['Genre'] = df_anime['Genre'].fillna("['unknown']")

df_anime['Genre'] = df_anime['Genre'].str.replace('[', '')
df_anime['Genre'] = df_anime['Genre'].str.replace(']', '')
df_anime['Genre'] = df_anime['Genre'].str.replace("'", '')

<ipython-input-332-1302959a843d>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_anime['Genre'] = df_anime['Genre'].str.replace('[', '')
<ipython-input-332-1302959a843d>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_anime['Genre'] = df_anime['Genre'].str.replace(']', '')


In [333]:
dummies = df_anime["Genre"].str.get_dummies(', ').add_prefix('genre_')
df_anime = pd.concat([df_anime, dummies], axis=1)

In [334]:
df_anime = df_anime.drop('Genre', axis=1)

In [335]:
df_anime

,Title,Synopsis,Type,Producer,Studio,Rating,genre_Action,genre_Adventure,genre_Cars,genre_Comedy,...,genre_Slice of Life,genre_Space,genre_Sports,genre_Super Power,genre_Supernatural,genre_Thriller,genre_Vampire,genre_Yaoi,genre_Yuri,genre_unknown
0,Cowboy Bebop,"In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,1,1,0,1,...,0,1,0,0,0,0,0,0,0,0
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Witch Hunter Robin,Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Bouken Ou Beet,It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Okusama ga Seitokaichou!: Seitokaichou to Ofur...,Bundled with the limited edition ninth manga v...,OVA,NaN,['Seven'],6.89,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9996,Lupin Shanshei Pilot,unknown,Special,NaN,['TMS Entertainment'],5.69,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9997,Hibike! Euphonium 2,Following their success in the qualifying roun...,TV,"['Lantis', 'Pony Canyon', 'Rakuonsha']",['Kyoto Animation'],8.29,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,Hibike! Euphonium Movie 1: Kitauji Koukou Suis...,After swearing off music due to an incident at...,Movie,"['Lantis', 'Shochiku']",['Kyoto Animation'],7.45,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Column Producer

In [336]:
df_anime['Producer'] = df_anime['Producer'].fillna("['unknown']")
df_anime['Producer'] = df_anime['Producer'].str.replace('[', '')
df_anime['Producer'] = df_anime['Producer'].str.replace(']', '')
df_anime['Producer'] = df_anime['Producer'].str.replace("'", '')

<ipython-input-336-d1b46fef72ac>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_anime['Producer'] = df_anime['Producer'].str.replace('[', '')
<ipython-input-336-d1b46fef72ac>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_anime['Producer'] = df_anime['Producer'].str.replace(']', '')


In [337]:
dummies = df_anime["Producer"].str.get_dummies(', ').add_prefix('producer_')
df_anime = pd.concat([df_anime, dummies], axis=1)

In [338]:
df_anime = df_anime.drop('Producer', axis=1)

In [339]:
df_anime

,Title,Synopsis,Type,Studio,Rating,genre_Action,genre_Adventure,genre_Cars,genre_Comedy,genre_Dementia,...,producer_feng,producer_flying DOG,producer_gimik,producer_i0+,producer_iQIYI,producer_indigo line,producer_m.o.e.,producer_teamKG,producer_tsuritama partners,producer_unknown
0,Cowboy Bebop,"In the year 2071, humanity has colonized sever...",TV,['Sunrise'],8.81,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...",Movie,['Bones'],8.41,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...",TV,['Madhouse'],8.31,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Witch Hunter Robin,Witches are individuals with special powers li...,TV,['Sunrise'],7.34,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bouken Ou Beet,It is the dark century and the people are suff...,TV,['Toei Animation'],7.04,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Okusama ga Seitokaichou!: Seitokaichou to Ofur...,Bundled with the limited edition ninth manga v...,OVA,['Seven'],6.89,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
9996,Lupin Shanshei Pilot,unknown,Special,['TMS Entertainment'],5.69,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
9997,Hibike! Euphonium 2,Following their success in the qualifying roun...,TV,['Kyoto Animation'],8.29,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,Hibike! Euphonium Movie 1: Kitauji Koukou Suis...,After swearing off music due to an incident at...,Movie,['Kyoto Animation'],7.45,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Column Studio

In [340]:
df_anime['Studio'] = df_anime['Studio'].fillna("['unknown']")
df_anime['Studio'] = df_anime['Studio'].str.replace('[', '')
df_anime['Studio'] = df_anime['Studio'].str.replace(']', '')
df_anime['Studio'] = df_anime['Studio'].str.replace("'", '')

<ipython-input-340-38ffa96b9cf9>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_anime['Studio'] = df_anime['Studio'].str.replace('[', '')
<ipython-input-340-38ffa96b9cf9>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_anime['Studio'] = df_anime['Studio'].str.replace(']', '')


In [341]:
dummies = df_anime["Studio"].str.get_dummies(', ').add_prefix('studio_')
df_anime = pd.concat([df_anime, dummies], axis=1)
df_anime = df_anime.drop('Studio', axis=1)

# Column Type

In [342]:
dummies = df_anime["Type"].str.get_dummies(', ').add_prefix('type_')
df_anime = pd.concat([df_anime, dummies], axis=1)
df_anime = df_anime.drop('Type', axis=1)

# Column Synopsis

In [343]:
#df_synospsis['Synopsis'] = df_synospsis['Synopsis'].apply(preprocess_text)

In [344]:
df_temp = df_anime[['Title', 'Synopsis']]

In [345]:
rem_nombre = r'\d+'
df_temp["Synopsis"] = df_temp["Synopsis"].apply(lambda x: re.sub(r'\s*(\r\n\s*)+|\[Written by .*?\]+|\(Source: .*?\)\s*', '', x))
df_temp["Synopsis"] = df_temp["Synopsis"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
df_temp = df_temp.apply(lambda x : x.astype(str).str.lower())

<ipython-input-345-0128e2d09aee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["Synopsis"] = df_temp["Synopsis"].apply(lambda x: re.sub(r'\s*(\r\n\s*)+|\[Written by .*?\]+|\(Source: .*?\)\s*', '', x))
<ipython-input-345-0128e2d09aee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["Synopsis"] = df_temp["Synopsis"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))


In [346]:
df_temp["Synopsis"] = df_temp["Synopsis"].apply(lambda x : word_tokenize(x))

In [347]:
df_temp["Synopsis"] = df_temp["Synopsis"].apply(lambda words : [word for word in words if word not in nltk.corpus.stopwords.words('english')])

In [348]:
model_wc = Word2Vec(df_temp["Synopsis"], vector_size=100, window=5, min_count=1, workers=4)

In [349]:
def get_vector(text):
    tokens = [token for token in text]
    vectors = [model_wc.wv[token] for token in tokens if token in model_wc.wv]
    return sum(vectors)/len(vectors)

In [350]:
df_temp["Synopsis"] = df_temp["Synopsis"].apply(get_vector)
df_anime["Synopsis"] = df_temp["Synopsis"]

In [351]:
df_anime['Synopsis'] = df_anime['Synopsis'].apply(lambda x : x.tolist())

# Column Title

In [352]:
df_temp["Title"] = df_temp["Title"].apply(lambda x: re.sub(r'\s*(\r\n\s*)+|\[Written by .*?\]+|\(Source: .*?\)\s*', '', x))
df_temp["Title"] = df_temp["Title"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

In [353]:
df_temp["Title"] = df_temp["Title"].apply(lambda x : word_tokenize(x))

In [354]:
df_temp["Title"] = df_temp["Title"].apply(lambda words : [word for word in words if word not in nltk.corpus.stopwords.words('english')])

In [355]:
model_wc = Word2Vec(df_temp['Title'], vector_size=100, window=5, min_count=1, workers=4)

In [356]:
df_temp

,Title,Synopsis
0,"[cowboy, bebop]","[-0.1705861, 0.50145656, 0.06242112, -0.050267..."
1,"[cowboy, bebop, tengoku, tobira]","[-0.123448856, 0.45442343, 0.04985982, -0.0083..."
2,[trigun],"[-0.1231289, 0.40324292, 0.04441019, -0.024013..."
3,"[witch, hunter, robin]","[-0.19546089, 0.57900697, 0.09218314, -0.04663..."
4,"[bouken, ou, beet]","[-0.14195767, 0.5947782, 0.07423981, -0.052870..."
...,...,...
9995,"[okusama, ga, seitokaichou, seitokaichou, ofur...","[-0.6446764, 0.9593443, 0.37641963, -0.3793349..."
9996,"[lupin, shanshei, pilot]","[-0.25085032, 0.76535547, 0.11640527, -0.10415..."
9997,"[hibike, euphonium, 2]","[-0.16635594, 0.50608194, 0.0073412308, 0.0332..."
9998,"[hibike, euphonium, movie, 1, kitauji, koukou,...","[-0.17828332, 0.53048754, -0.011683915, 0.0508..."


In [357]:
def get_title_vector(title):
    words = title.lower().split()
    vectors = [model_wc.wv[word] for word in words if word in model_wc.wv.key_to_index]
    if len(vectors) > 0:
        return sum(vectors) / len(vectors)
    else:
        return np.zeros(model_wc.vector_size)

In [358]:
df_anime["Title"] = df_anime["Title"].apply(get_title_vector)

In [359]:
scaler = MinMaxScaler()

In [360]:
df_anime["Title"]

0       [-0.0021524713, 0.0053312276, 0.003411077, 0.0...
1       [-0.003455235, 0.0027009884, 0.0041536717, 0.0...
2       [-0.006152878, -4.0222578e-05, 0.006970724, 0....
3       [0.007513949, 0.0018292841, 0.0019320678, -0.0...
4       [0.0009120815, 0.002911451, -0.0029200714, -0....
                              ...                        
9995    [-0.002785618, 0.0016222183, -0.0022423512, -0...
9996    [-0.0027614385, -0.008291604, -0.0039979913, 0...
9997    [-0.011072465, 0.008716099, 0.004820809, -0.00...
9998    [0.00028179897, 0.008329985, 0.005740296, 0.00...
9999    [-0.0023573472, 0.0001129106, 0.0020167718, -0...
Name: Title, Length: 9986, dtype: object

In [361]:
df_anime["Title"] = scaler.fit_transform(df_anime["Title"].tolist())

In [362]:
df_anime

,Title,Synopsis
0,"[cowboy, bebop]","[-0.1705861, 0.50145656, 0.06242112, -0.050267..."
1,"[cowboy, bebop, tengoku, tobira]","[-0.123448856, 0.45442343, 0.04985982, -0.0083..."
2,[trigun],"[-0.1231289, 0.40324292, 0.04441019, -0.024013..."
3,"[witch, hunter, robin]","[-0.19546089, 0.57900697, 0.09218314, -0.04663..."
4,"[bouken, ou, beet]","[-0.14195767, 0.5947782, 0.07423981, -0.052870..."
...,...,...
9995,"[okusama, ga, seitokaichou, seitokaichou, ofur...","[-0.6446764, 0.9593443, 0.37641963, -0.3793349..."
9996,"[lupin, shanshei, pilot]","[-0.25085032, 0.76535547, 0.11640527, -0.10415..."
9997,"[hibike, euphonium, 2]","[-0.16635594, 0.50608194, 0.0073412308, 0.0332..."
9998,"[hibike, euphonium, movie, 1, kitauji, koukou,...","[-0.17828332, 0.53048754, -0.011683915, 0.0508..."


# Output

In [363]:
df_anime.to_csv('output_data_anime.csv')

In [364]:
#nltk.download("stopwords")

#stopwords = set(stopwords.words('english'))
#words = [word for word in df_anime["Synopsis"] if not word in stopwords]
#df_anime["Synopsis"] = df_anime["Synopsis"].apply(lambda x: ' '.join([word for word in x.translate(str.maketrans('', '', string.punctuation)).lower().split() if word not in stopwords]))

In [365]:
#nltk.download('punkt')

#df_anime["Synopsis"] = df_anime["Synopsis"].astype("str")
#df_anime["Synopsis"] = df_anime["Synopsis"].apply(lambda x : word_tokenize(x))

In [366]:
#df_anime["Synopsis"] = df_anime["Synopsis"].apply(lambda x: ' '.join(x))

#vectorizer = CountVectorizer(stop_words='english')
#vectors = vectorizer.fit_transform(df_anime["Synopsis"])

In [367]:
#vocab = vectorizer.vocabulary_
#counts = vectors.sum(axis=0)

# Créer un DataFrame à partir du vocabulaire et des comptages
#df_vocab = pd.DataFrame({'mot': list(vocab.keys()), 'compte': counts.tolist()[0]})


In [368]:
#df_vocab = df_vocab.sort_values('compte', ascending=False).reset_index(drop=True)

In [369]:
#df_vocab